In [472]:
import sys
import os
import mmh3
import numpy as np
import itertools
import collections
import pandas as pd
import more_itertools
from itertools import tee
import re
#################### Utilities ######################
#hashes a list of strings
def listhash(l,seed):
	val = 0
	for e in l:
		val = val ^ mmh3.hash(e, seed)
	return val 

################### Similarity ######################

docs = {} #dictionary mapping document id to document contents

data = pd.read_csv(r"C:\Users\hasee\Documents\comptools\DTU_CTDS\data\articles1.csv")

data1 = data[:50000]
key = data1["Unnamed: 0"]
data2 = data1["content"]
docs = data2.to_dict()
# docs = data2.tolist()
# g_docs = data2.tolist()
docs = list(zip(data2.tolist())) ## convert to tuple for starmap to work basically getting ['dfdf', 'dddd', 'sss'] => [('dfdf'), ('dddd'), ('sss')]
# print(docs[2])


In [473]:
''' Extension of standard pairwise function to 3-pairwise from the py standard lib'''
def three_pairwise(iterable):
    # pairwise('ABCDEFG') --> AB BC CD DE EF FG
    # three_pairwise('ABCDEFG') --> ABC BCD CDE DEF EFG FG
    a, b, c = tee(iterable, 3) ## Create three iterators
    next(b, None) ## Advance the second
    next(c, None) ## Advance the third
    next(c, None) ## Advance the third once again. This ensures the third iterator starts at the third element and that we can create the 3-tuple
    return zip(a, b, c) ### Zip everything (1 elem, 2 elem, and 3 elem) concurrently.

def hash_docs(doc, k=50):
    # three_tup=zip
    lst_shingles_h=[]
    # for doc in docs:
    seed=1
    three_tup=three_pairwise(doc.split(" "))

    three_tup=list(three_tup)

    three_tup=[elem for elem in three_tup if '' not in elem]
    
    # for t in three_tup:
    #     if '' or ' ' in t:
    #         print("single q", t)
    #     if " " or "" in t:
    #         print("double ", t)

    # print(set(three_tup))

    # seeds = [seed+i for i in range(0,k)]
    ret_lst=[]
    # for s in seeds:  
    #     ret_lst.append(np.array(min([listhash(shingle, s) for shingle in three_tup], default=0)))  
    # lens=[]
    # my_lst=[]
    # for shingle in three_tup:
    #     my_lst.append(listhash(shingle, seed))
    #     lens.append(len(my_lst))
    # ret_lst.append(np.array(sorted(my_lst)[:k]))
    # print("min len", min(lens))
    # ret_lst.append(np.array(sorted([listhash(shingle, seed) for shingle in three_tup])[:k]))
    # print("min leng", min(lens))
    
    # s_lst = sorted([listhash(shingle, seed) for shingle in three_tup])
    # print(len(s_lst))
    return np.array(sorted([listhash(shingle, seed) for shingle in three_tup]))##[:k])#, len(s_lst)

''' Returns document as a list of hashes'''
'''
Creates shingles of size q, removing shingles of size < q. Removes duplicates and hashes the result.
'''
def hashed_lst_shingles(my_docs):


    hashed_docs = itertools.starmap(hash_docs, my_docs)
    return hashed_docs

# doc = "You and me, we made a vow. For better or for worse. I can't believe you let me down"
# lst_shnigles = hashed_lst_shingles(3, doc)
# print(lst_shnigles)
# [['You', 'and', 'me,'], ['and', 'me,', 'we'], ['me,', 'we', 'made'], ['we', 'made', 'a'], ['made', 'a', 'vow.'], ['a', 'vow.', 'For'], ['vow.', 'For', 'better'], ['For', 'better', 'or'], ['better', 'or', 'for'], ['or', 'for', 'worse.'], ['for', 'worse.', 'I'], ['worse.', 'I', "can't"], ['I', "can't", 'believe'], ["can't", 'believe', 'you'], ['believe', 'you', 'let'], ['you', 'let', 'me'], ['let', 'me', 'down']]

### MinHashing implementation

In [474]:
# test_docs=list(zip(["You and me, we made a vow. For better or for worse. I can't believe you let me down",
# "Time, space and state. Equal everything explanable.",
# "You and me, we made a vow. For better or for worse. I can't believe you let me down"]))
hashed_shingles=hashed_lst_shingles(docs)
# print(hash)
hashed_shingles = list(hashed_shingles)
hashed_shingles=[lst for lst in hashed_shingles if len(lst)>100]
# hashed_shingles
sig_m=np.array(hashed_shingles).T
print(sig_m.shape)


(47521,)


C:\Users\hasee\AppData\Local\Temp\ipykernel_12596\928616.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sig_m=np.array(hashed_shingles).T


In [475]:
lens=min([len(x) for x in sig_m])
print(lens)
num_docs=sig_m.shape[0]
for i in range(num_docs):
    sig_m[i]=sig_m[i][:lens]
sig_m=np.stack(sig_m).T
sig_m.shape


101


(101, 47521)

In [476]:
# my_s=[]
# for i in range(len(hashed_shingles)):
#     my_s.append(hashed_shingles[i][1])
# min(my_s)
# # min(my_s)

### LSH implementation

In [477]:
''' Jaccard similarity'''
def jaccard(s1, s2):
    return len(s1 & s2) / len(s1 | s2)

''' Implementation of LSH, dividing signature matrix into b band with r rows each'''
def LSH(sig_m, b, r):
    b=20
    r=5
    #b*r=num_hash_funcs

    sim_hashes=[]
    start=0
    for i in range(b):
        sim_hashes.append([listhash(col, seed=i) for col in sig_m[start:start+r,:].T])
        start=i+r

    return sim_hashes

''' Find candidate pairs by checking to see if the hashes match.
Then we check to see that the Jaccard similarity b/w each pair of docs is atleast t. If so, we consider it a candidate pair otherwise not '''
def get_cand_pairs(sim_hashes, t):
    cand_pairs=set()
    for L in sim_hashes:
        dups = collections.defaultdict(list)
        for i, e in enumerate(L):
            dups[e].append(i)
        for _, v in sorted(dups.items()):
            if len(v) >= 2:
                cand_pairs.add(tuple(v))
    cand_pairs=list(cand_pairs)
    filtered_cand_pairs = [pair for pair in cand_pairs if (jaccard(set(sig_m[:, pair[0]]), set(sig_m[:, pair[1]])) > t)]
    return filtered_cand_pairs
   

In [478]:
b=20
r=5
sim_hashes = LSH(sig_m, b=20, r=5)
pairs=get_cand_pairs(sim_hashes, t=(1/b)**(1/r))
len(pairs)
# pairs

31

In [479]:
sig_m.shape

(101, 47521)

In [480]:
pairs

[(47381, 47422),
 (42957, 42965),
 (43171, 43321),
 (45465, 45663),
 (41980, 42274),
 (15705, 24350),
 (44545, 44740),
 (15236, 17272),
 (47032, 47046),
 (41620, 41633),
 (40643, 40763),
 (44249, 44435),
 (47384, 47518),
 (38982, 38995),
 (43546, 43845, 44135),
 (38765, 38816),
 (45476, 45664),
 (19563, 29213),
 (45790, 45997),
 (41101, 41187),
 (46647, 46703),
 (41786, 47187),
 (10977, 12007),
 (8035, 30924),
 (16620, 44519),
 (46331, 46346, 46704),
 (41731, 41980, 42274),
 (46330, 46345),
 (41619, 41632, 41979, 47031),
 (43546, 44135),
 (47380, 47421)]

In [483]:
docs[42957]
# docs[47422]

('Take a look at 38 photos of the week from July 17 through July 23.',)

In [484]:
docs[42965]

('New York (CNN) Donald Trump lashed out Wednesday at a female lawyer whom he had called ”disgusting” several years ago for requesting a break to pump breast milk in the middle of a deposition, calling her a ”vicious, horrible person.” In an interview with CNN’s Dana Bash, the Republican presidential candidate claimed that the attorney, Elizabeth Beck, had wanted to pump breast milk in other people’s presence. Trump alleged that Beck was now retaliating against him because she was unhappy about having lost that legal battle, and that she had fabricated a story about Trump losing his cool and storming out of the deposition room.  ”She wanted to breast pump in front of me and I may have said that’s disgusting, I may have said something else. I thought it was terrible,” Trump said. ”She’s a vicious, horrible person.” Elizabeth Beck described the 2011 encounter    first reported by the New York Times late Tuesday    as Trump having ”an absolute meltdown” in an interview with CNN’s ”New Day

As can be seen using LSH, we find that documents 5 and 6 are similar.